# QC of FASTQ runs

This Jupyter notebook allows you to perform some basic quality control analysis of reads for a given SRA accession. Quality checks are computed by the program [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) and the output data is parsed by python functions in this notebook. 

## 0. Setting up
We will check for dependencies during this step and ensure that all necessary applications are installed and in place. 

In [1]:
from qc_functions import *
from IPython.display import display, Markdown

In [2]:
%%bash
## check if java is installed
java --version > /dev/null
if [ $? -ne 0 ]; then
    echo "Java is missing. Please install Java to proceed."
fi

unzip -hh > /dev/null
if [ $? -ne 0 ]; then
    echo "unzip is missing. Please install unzip to proceed."
fi

In [3]:
%%bash
## Download and install fastqc
if [ ! -f ./FastQC/fastqc ]; then
    wget -q https://www.bioinformatics.babraham.ac.uk/projects/fastqc/fastqc_v0.11.8.zip
    unzip -q fastqc_v*.zip
    chmod a+x ./FastQC/fastqc
    rm fastqc_v*.zip
fi

## Download and install sra-toolkit
if [ ! -f ./sratoolkit.*-ubuntu64/bin/fastq-dump ]; then
    wget -q https://ftp-trace.ncbi.nlm.nih.gov/sra/sdk/current/sratoolkit.current-ubuntu64.tar.gz
    tar -xzf sratoolkit.current-ubuntu64.tar.gz
    rm sratoolkit.current-ubuntu64.tar.gz
fi

## 1. Input SRA accessions

In the cell below, enter the SRA run accession numbers for which you would like to obtain QC results. Multiple run accessions can be in multiple lines, or separated by a comma or space. 

In [4]:
sra_accs = """
    DRR042075,
    DRR019508
    
"""

## 2. Fetch reads from SRA and run FastQC

In this step we will parse the list of SRA run accessions, download the reads in FASTQ format from SRA and run FastQC on those reads. 

In [5]:
sra_accs = parse_sra_accs(sra_accs)

print("The list of SRA accessions provided is: \n{}". format(sra_accs))

The list of SRA accessions provided is: 
DRR042075 DRR019508


In [6]:
%%bash -s "$sra_accs"

FASTQ_DUMP="./sratoolkit.*-ubuntu64/bin/fastq-dump"
FASTQC="./FastQC/fastqc"

for acc in $1; do
    echo "Fetching ${acc} reads in FASTQ format from SRA..." ;
    $FASTQ_DUMP -A ${acc} --gzip -N 1 -X 10000 ;

    echo "Running FASTQC on ${acc} reads..." ;
    $FASTQC --extract -q ${acc}.fastq.gz ;
done

echo "Deleting all fastq files..."
rm *.fastq.gz 

Fetching DRR042075 reads in FASTQ format from SRA...
Read 10000 spots for DRR042075
Written 10000 spots for DRR042075
Running FASTQC on DRR042075 reads...
Fetching DRR019508 reads in FASTQ format from SRA...
Read 10000 spots for DRR019508
Written 10000 spots for DRR019508
Running FASTQC on DRR019508 reads...
Deleting all fastq files...


2019-04-24T18:26:25 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768
2019-04-24T18:26:25 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768
2019-04-24T18:26:25 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768
2019-04-24T18:26:26 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768
2019-04-24T18:26:26 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768
2019-04-24T18:26:29 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768
2019-04-24T18:26:30 fastq-dump.2.9.6 warn: block-size in local file 131072 does not match requested value 32768


## 3. Inspect QC data

In this step, we will parse the FastQC output and generate a table with pertinent data to inspect. Click on the 'Report' link in the table to view the entire FastQC report. 

## 3a. [Optional] Change default parameters
There are two parameters that you can make changes to. By default, we have picked sensibles cut-offs but feel free to change them to your liking. 

1. `qc_level` can be either 'FAIL' or 'WARN'. By default, it is 'FAIL', meaning a list of all metrics that have failed will be displayed in the table. If you choose 'WARN', both failed metrics and the ones with warnings will be shown in the table. 
2. `threshold` is the minimum quality score for the read that is acceptable. By default, this value is set to 27. The percentage of reads with overall quality greater than the `threshold` is shown in the table. 

In [7]:
## Edit the following values if you want to change the defaults

qc_level = 'fail'  ## can be 'fail' or 'warn'
threshold = 27 ## must be a number below 40 

In [8]:
tbl_str = generate_results_table_md(sra_accs, qc_level, threshold)
display(Markdown(tbl_str))

At least 90% of the reads have quality scores over the threshold in the following accessions: DRR042075, DRR019508


|SRA Acc.|No. of reads|Read length|Percent GC|Poor qual reads|Failed metrics|Pct reads over threshold qual|FastQC Report|
|----|----|----|----|----|----|----|----|
|DRR042075|10000|287-301|48|0|Per base sequence quality<br>Per base sequence content<br>Per sequence GC content|99.71|<a href="./DRR042075_fastqc.html" target="_blank">Report</a> |
|DRR019508|10000|86-502|45|0|Per base sequence quality<br>Per sequence GC content|99.94|<a href="./DRR019508_fastqc.html" target="_blank">Report</a> |


In [9]:
results_tbl = generate_results_table(sra_accs, qc_level, threshold)

At least 90% of the reads have quality scores over the threshold in the following accessions: DRR042075, DRR019508


In [17]:
results_tbl

defaultdict(list,
            {'SRA Acc': ['DRR042075', 'DRR019508'],
             'Total Sequences': ['10000', '10000'],
             'Sequence length': ['287-301', '86-502'],
             '%GC': ['48', '45'],
             'Sequences flagged as poor quality': ['0', '0'],
             'Failed metrics': ['Per base sequence quality\nPer base sequence content\nPer sequence GC content',
              'Per base sequence quality\nPer sequence GC content'],
             'Pct reads over threshold qual': ['99.71|', '99.94|'],
             'Failed Metrics': [],
             0: []})

['Per base sequence quality\nPer base sequence content\nPer sequence GC content',
 'Per base sequence quality\nPer sequence GC content']

In [16]:
import pandas as pd
d = pd.DataFrame.from_dict(results_tbl).set_index('SRA Acc', drop=True)


ValueError: arrays must all be same length